In [315]:
import random as rnd
import numpy as np
from typing import Callable
import xlsxwriter as xw

In [316]:
class Item:
    def __init__(self, peso, volume, nome):
        self.peso = peso
        self.volume = volume
        self.nome = nome

In [317]:
# definição do material genético. Os genes são sequências de 0s ou 1s representando se o item está na bagagem ou não
class Genoma:
    def __init__(self, pertinencia:list[int]):
        self.gene = pertinencia

In [318]:
# gero uma população com "n" indivíduos aleatórios no espaço x_space[0] <= x< <= x_space[1] e y_space[0] <= y <= y_space[1]
def gerar_populacao(n:int, coisas:list[Item]) -> list[Genoma]:
    return [Genoma(
        [rnd.randint(0,1) for _ in range(len(coisas))] # lista com a quantidade de bits igual a quantidade de coisas.
    ) for _ in range(n)]                               # a quantidade de genes é especificada por n.

In [319]:
def cruz_troca(g1:Genoma, g2:Genoma) -> tuple[Genoma,Genoma]:
    length = len(g1.gene)
    if length < 2:
        return g1, g2

    p = rnd.randint(1, length - 1)
    return Genoma(g1.gene[0:p] + g2.gene[p:]), Genoma(g2.gene[0:p] + g1.gene[p:])

In [320]:
def mutar(g:Genoma) -> Genoma:
    G = Genoma(g.gene)
    for _ in range(int(0.2*len(G.gene))):
        index = rnd.randrange(len(G.gene))
        G.gene[index] = abs(G.gene[index] - 1)
    return G

In [321]:
# esta função selecionará um par de genomas, dando maior prioridade de escolha àqueles com o menor valor da função objetivo.
def selec_par_melhores(func: Callable[[Genoma, float, float, list[Item]], float], pop: list[Genoma], peso_max:float, vol_max:float, coisas:list[Item]) -> list[Genoma]:
    weights = [func(indv, peso_max, vol_max, coisas) for indv in pop] # calcula a funcao objetivo para os genomas
    M = max(weights)
    m = min(weights)
    if m == M:
        return [pop[0], pop[-1]]
    weights = list(map(lambda x: (M - x)/(M - m), weights)) # mapeia os valores da função objetivo
    # para pesos, sendo o maior valor da func sendo 0 e o menor sendo 1, já que queremos que genomas com valores menores da func
    # tenham pesos maiores.
    return rnd.choices(population=pop,  # seleciona 2 indivíduos da população
                       weights=weights, # pesados pelo seu score
                       k=2)

In [322]:
def func_objetivo(g:Genoma, peso_max:float, vol_max:float, coisas:list[Item]) -> float:
    peso = 0
    vol = 0
    for i, item in enumerate(coisas):
        peso += item.peso if g.gene[i] == 1 else 0
        vol += item.volume if g.gene[i] == 1 else 0
    if peso > peso_max or vol > vol_max:
        return peso_max + vol_max
    else:
        return peso_max + vol_max - peso - vol

In [323]:
def alg_genetico(
		geradora_de_pop: Callable[[int, list[Item]], list[Genoma]],
		n_indv:          int,
		taxa_mut:        float,
		taxa_cruz:       float,
		n_elite:         int,
		coisas:          list[Item],
		peso_max:        float,
		vol_max:         float,
		func_obj:        Callable[[Genoma, float, float, list[Item]], float],
		tol:             float,
		func_selecao:    Callable[[ Callable[[Genoma, float, float, list[Item]], float], list[Genoma], float, float, list[Item]], list[Genoma]],
		func_cruzamento: Callable[[Genoma, Genoma], tuple[Genoma, Genoma]],
		func_mutacao:    Callable[[Genoma], Genoma],
		max_iter:        int
) -> tuple[list[Genoma], int]:
	
	pop = geradora_de_pop(n_indv, coisas) # gerar uma população inicial
	pop = sorted(pop, key=lambda g: func_obj(g, peso_max, vol_max, coisas)) # ordenar os genomas para que aqueles com o menor valor de funcao objetivo apareçam primeiro
	prev_best = sum([func_obj(indv, peso_max, vol_max, coisas) for indv in pop]) + tol + tol # impedir que o algoritmo convirja na primeira iteração

	for i in range(max_iter):
		if abs(prev_best - sum([func_obj(indv, peso_max, vol_max, coisas) for indv in pop])) <= tol: # se a diferença entre 2 iterações seguidas for melhor que uma tolerância, parar o algoritmo.
			return (pop, i)
		
		prev_best = sum([func_obj(indv, peso_max, vol_max, coisas) for indv in pop])
		prox_pop = pop[:n_elite] # incluir a elite inalterada na proxima geração

		for j in range((len(pop) - n_elite + 1) // 2):
			pais = func_selecao(func_obj, pop, peso_max, vol_max, coisas) # selecionar dois individuos

			prob = rnd.uniform(0, 1)
			if prob <= taxa_cruz: # se o numero aleatorio for maior que a taxa de cruzamento, cruzar.
				pais[0], pais[1] = func_cruzamento(pais[0], pais[1])

			# para cada resultante, verificar se teremos mutação.
			prob = rnd.uniform(0, 1)
			if prob <= taxa_mut:
				pais[0] = func_mutacao(pais[0])
			prob = rnd.uniform(0, 1)
			if prob <= taxa_mut:
				pais[1] = func_mutacao(pais[1])
			
			prox_pop += pais # adicionar os novos indvíduos à nova população
		
		if (len(pop) - n_elite) % 2 != 0: # para o caso de o número de indivíduos que devemos gerar não ser divisível por 2, eliminar o último
			prox_pop = prox_pop[:-1]      # já que estaremos gerando 1 a mais do que o necessário neste caso.
		
		pop = prox_pop
		pop = sorted(pop, key=lambda g: func_obj(g, peso_max, vol_max, coisas))

	return (pop, max_iter)

### teste base

In [335]:
coisas = [
    Item(1, 1, "farinha"),
    Item(0.5, 0.25, "laptop"),
    Item(2, 3, "camisas"),
    Item(3, 3, "calças"),
    Item(3, 4, "lençóis"),
    Item(0.25, 0.5, "acessóris"),
    Item(1.5, 1, "vinho"),
]

peso_tot = 0
vol_tot = 0
for item in coisas:
    peso_tot += item.peso
    vol_tot += item.volume

print("peso total:", peso_tot, "volume_total:", vol_tot)

peso_max = 6
vol_max = 10

pop, n_iter = alg_genetico(
    gerar_populacao,
    30,
    0.7,
    0.7,
    1,
    coisas,
    peso_max,
    vol_max,
    func_objetivo,
    0.01,
    selec_par_melhores,
    cruz_troca,
    mutar,
    1000
)

print("iter:", n_iter)
peso_tot = 0
vol_tot = 0
item_list = []
for i, pert in enumerate(pop[0].gene):
    if pert == 1:
        item_list.append(coisas[i].nome)
        peso_tot += coisas[i].peso
        vol_tot += coisas[i].volume
print("peso total:", peso_tot, "volume_total:", vol_tot, "\ncoisas:")
for item in item_list:
    print(item, sep=', ')

peso total: 11.25 volume_total: 12.75
iter: 173
peso total: 6 volume_total: 8 
coisas:
farinha
camisas
lençóis
